In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from qlora import *
from collections import defaultdict
import copy
import json
from os.path import exists, join, isdir
from dataclasses import dataclass, field
import sys
from typing import Optional, Dict, Sequence
import numpy as np
from tqdm import tqdm
import logging
import bitsandbytes as bnb
import pandas as pd
import importlib
from packaging import version
from packaging.version import parse
import warnings
from sklearn.metrics.pairwise import manhattan_distances
from torchmetrics.functional.pairwise import pairwise_manhattan_distance as manhattan
from torchmetrics.functional.pairwise import pairwise_cosine_similarity as cossim
import numpy as np

import torch
import transformers
from torch.nn.utils.rnn import pad_sequence
import argparse
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    set_seed,
    Seq2SeqTrainer,
    BitsAndBytesConfig,
    LlamaTokenizer

)
from datasets import load_dataset, Dataset, load_from_disk
import evaluate

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)
from peft.tuners.lora import LoraLayer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

from transformers.modeling_utils import unwrap_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES
from transformers.utils import is_peft_available
from peft import PeftModel

/mnt/data/sonia/miniconda3/envs/llama2/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
argdict = {
  'model_name_or_path' : '/mnt/data/zoo/llama2/llama2-7b-hf/',
  'multihead' : 4,
  'use_auth' : True,
  'output_dir' : '/mnt/data/sonia/ckpts/deeeebug',
  'logging_steps' : 10 ,
  'save_strategy' : 'steps',
  'data_seed' : 42 ,
  'save_steps' : 5 ,
  'save_total_limit' : 40 ,
  'evaluation_strategy' : 'steps' ,
  'eval_dataset_size' : 5 ,
  'max_eval_samples' : 100 ,
  'per_device_eval_batch_size' : 1 ,
  'max_new_tokens' : 60 ,
  'dataloader_num_workers' : 1 ,
  'group_by_length' : True,
  'logging_strategy' : 'steps' ,
  'remove_unused_columns' : False ,
  'do_train' : True ,
  'eval_samples' : True ,
  'do_mmlu_eval' : False ,
  'diversity' : False ,
  'divdist' : 'manhattan' ,
  'lora_r' : 64 ,
  'lora_alpha' : 16 ,
  'lora_modules' : 'all' ,
  'double_quant' : True,
  'quant_type' : 'nf4' ,
  'bf16' : True,
  'bits' : 4 ,
  'warmup_ratio' : 0.03 ,
  'lr_scheduler_type' : 'constant' ,
  'gradient_checkpointing' : True,
  'dataset' : '/mnt/data/sonia/honeygan/cloze_apr13.dat',
  'source_max_len' : 60 ,
  'target_max_len' : 60 ,
  'per_device_train_batch_size' : 1 ,
  'gradient_accumulation_steps' : 16 ,
  'max_steps' : 60 ,
  'eval_steps' : 1 ,
  'learning_rate' : 0.0002 ,
  'adam_beta2' : 0.999 ,
  'max_grad_norm' : 0.3 ,
  'lora_dropout' : 0.1 ,
  'weight_decay' : 0.0 ,
  'seed' : 0
}

arglist = [f'--{k}={v}' for k,v in argdict.items()]

In [3]:
hfparser = transformers.HfArgumentParser((
    ModelArguments, DataArguments, TrainingArguments, GenerationArguments
))
model_args, data_args, training_args, generation_args  = hfparser.parse_args_into_dataclasses(args=arglist, return_remaining_strings=True)[:-1]
training_args.generation_config = transformers.GenerationConfig(**vars(generation_args))
args = argparse.Namespace(
    **vars(model_args), **vars(data_args), **vars(training_args)
)

In [7]:
checkpoint_dir, completed_training = get_last_checkpoint(args.output_dir)
model, tokenizer = get_accelerate_model(args, checkpoint_dir)
model.config.use_cache = False
    
print('loaded model')
set_seed(args.seed)

data_module = make_data_module(tokenizer=tokenizer, args=args)

loading base model /mnt/data/zoo/llama2/llama2-7b-hf/...
MultiheadLlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layerno

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


here
Adding special tokens.
adding LoRA modules...
['gate_proj', 'v_proj', 'k_proj', 'down_proj', 'up_proj', 'q_proj', 'o_proj']
loaded model
Splitting train dataset in train and validation according to `eval_dataset_size`


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/197485 [00:00<?, ? examples/s]

In [8]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MultiheadLlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (b

In [ ]:
trainerclass = Seq2SeqTrainer
trainer = trainerclass(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
)
class evalSampleCallback(transformers.TrainerCallback):
    def on_evaluate(self, args, state, control, model, **kwargs):
        trainer.model.eval()
        metrics = trainer.predict(test_dataset=data_module['eval_dataset'],metric_key_prefix="predict")
        
        predictions = []
        for i in range(len(metrics.predictions)):
            logit = metrics.predictions[i]
            label = metrics.label_ids[i] #just to see positions where prompt tokens are at
            logit_abcd = logit[label != IGNORE_INDEX]
            toks = np.argmax(logit_abcd, axis=1)
            predictions.append(
                ''.join(trainer.tokenizer.decode(toks, skip_special_tokens=True, clean_up_tokenization_spaces=True))
                )
        
        for pred in predictions:
            print(pred)
    
    
trainer.add_callback(evalSampleCallback)

In [ ]:
trainer.evaluate(metric_key_prefix="eval")

хідatticeaten betre vent replied compilesites compileлосо compileNotFoundръcció sixolare Va Aspлав Josephacional owstorageTouchलOT Proof chooseravits review {{Hozzáférés streets surg {{ groièrement可 tub searchedusztusièrement click Sue Sue Tit deutschen quoted Т believeULL continue continueutors Herzritt finalbatIcon
хідswapários safely ];rav pseudoExtra prime pseudo pseudo pseudoätten soldАziłccióलOT erreichteOT составляетpecially Nich surg erreichte Basicaspxlect surgneyüt可 tub.,usztusièrement Sue            Sue Titʷ Mitte Т believe Round altaографDirectory Frameworkgency Pelдрині copiedRIPTcció Гра Color
його його nostziłccióलOT IBiterrav WisconsinExtraкраїн Operationnahmen surgAssert gro illustrneyüt$$ल té可可可ièrement TABLE TABLE йогоounced deutschenlaps Тimportant deutschen variance Byteсленgoing gepubliceerd operators Влади operators dangerbmatrix Color
хідatticeaten betre ventflash compilesites compileMODE compileNotFound lap sched sixobierno Ha nostziłccióल choose choosedeutsch 

{'eval_loss': 12.69903564453125,
 'eval_runtime': 1.3771,
 'eval_samples_per_second': 3.631,
 'eval_steps_per_second': 3.631}

In [ ]:
metrics = trainer.predict(test_dataset=data_module['eval_dataset'],metric_key_prefix="predict")